# Executive Order Sentiment

In [1]:
import json
import os
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas

## Sentiment

https://arxiv.org/pdf/2312.06281

In [9]:
def build_stats(result):
    return {
        'input_tokens': result['raw'].usage_metadata['input_tokens'],
        'output_tokens': result['raw'].usage_metadata['output_tokens']
    }

In [ ]:
class EmotionScores(BaseModel):
    joy: int = Field(description="Joy conveyed on a scale of 1-10.")
    sadness: int = Field(description="sadness conveyed on a scale of 1-10.")
    fear: int = Field(description="fear conveyed on a scale of 1-10.")
    disgust: int = Field(description="disgust conveyed on a scale of 1-10.")
    surprise: int = Field(description="surprise conveyed on a scale of 1-10.")

class EmotionResponse(BaseModel):
    initial_scores: EmotionScores = Field(description="Initial emotion scores.")
    critique: str = Field(description="Critique of the initial scores.")
    revised_scores: EmotionScores = Field(description="Revised emotion scores.")

prompt = ChatPromptTemplate.from_template(
    '\n '.join([
        'Your task is to understand and score the emotions the president is conveying in the following executive order.',
        '- Does the president want the audience to feel joy, sadness, anger, fear, disgust, or surprise after reading the executive order? '
        '- For each emotion that should be conveyed, score the relative intensity on a scale of 1-10.',
        '- Critique your answer by thinking through it step by step.'
        '- Finally, give your revised scores.'
        '{text}',
    ])
)

def make_pipeline(model):
    return prompt | model.with_structured_output(EmotionResponse, include_raw=True)

In [2]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_retries=3
)

In [ ]:
source_dir = 'data/executive_orders/cleaned/'
target_dir = 'data/executive_orders/sentiment/'
log_path = 'data/executive_orders/sentiment_log.csv'

sentiment_pipeline = make_pipeline(gemini)

def process_eo(eo_path):
    stats = {
        'path': eo_path
    }

    start_time = time.time()
    try:
        with open(source_dir + eo_path) as source_file:
            eo = json.load(source_file)
            result = sentiment_pipeline.invoke({'text': '\n'.join(eo["content"])})
            stats = stats | build_stats(result)

        # write the cleaned eo to the target directory
        with open(target_dir + eo_path, 'w') as target_file:
            json.dump(result['parsed'].model_dump(), target_file, indent=4)
    except Exception as e:
        stats['error'] = str(e)

    duration = time.time() - start_time
    stats['duration'] = duration

    return stats

In [ ]:
concurrency = 8

eo_paths = os.listdir(source_dir)
eo_paths.sort()

results = []
with ThreadPoolExecutor(max_workers=concurrency) as executor:
    futures = [
        executor.submit(process_eo, path) for path in eo_paths
    ]

    for future in as_completed(futures):
        result = future.result()
        print(result)
        results.append(result)

pandas.DataFrame(results).to_csv(log_path, index=False)

{'path': 'executive-order-10000-regulations-governing-additional-compensation-and-credit-granted.json', 'input_tokens': 4825, 'output_tokens': 73, 'duration': 1.4616789817810059}
{'path': 'executive-order-10.json', 'input_tokens': 356, 'output_tokens': 84, 'duration': 1.4895906448364258}
{'path': 'executive-order-1.json', 'input_tokens': 314, 'output_tokens': 102, 'duration': 1.5081360340118408}
{'path': 'commission-appointing-commissioners-for-surveying-the-district-territory-for-the-permanent.json', 'input_tokens': 569, 'output_tokens': 94, 'duration': 1.5240669250488281}
{'path': 'executive-order-1000-alamo-national-forest-new-mexico.json', 'input_tokens': 1183, 'output_tokens': 110, 'duration': 1.6396288871765137}
{'path': 'executive-order-0.json', 'input_tokens': 314, 'output_tokens': 80, 'duration': 1.8411200046539307}
{'path': 'executive-order-10001-prescribing-or-amending-portions-the-selective-service-regulations.json', 'input_tokens': 21358, 'output_tokens': 92, 'duration': 1

## Political Bias

In [ ]:
class PoliticalBias(BaseModel):
    ideological_tilt: int = Field(description="-5 = Highly Conservative, 0 = Centrist/Moderate, +5 = Highly Liberal")
    immigration: int = Field(description="-5 = Strictly Restrictive (Closed), 0 = Balanced/Selective, +5 = Extremely Open")
    executive_power: int = Field(description="-5 = Strongly Restrains Executive Authority, 0 = Neutral/Balanced, +5 = Strongly Expands Executive Authority")
    federalism: int = Field(description="-5 = Heavy State Control, 0 = Cooperative Balance, +5 = Strong Federal Control")
    security_liberty: int = Field(description="-5 = Strong Liberties/Privacy Focus, 0 = Balanced Approach, +5 = Heavy Security Focus")
    economic_regulation: int = Field(description="-5 = Strong Deregulation/Laissez-Faire, 0 = Moderate Regulation, +5 = Heavy Government Intervention")
    social_inclusion: int = Field(description="-5 = Emphasis on Conformity/Traditional Uniformity, 0 = Neutral, +5 = Emphasis on Diversity/Pluralism")

class PoliticalBiasResponse(BaseModel):
    initial_scores: PoliticalBias = Field(description="Initial political bias scores.")
    critique: str = Field(description="Critique of the initial scores.")
    revised_scores: PoliticalBias = Field(description="Revised political bias scores.")

political_bias_prompt = ChatPromptTemplate.from_template(
    '\n '.join([
        'Your task is assess the political bias and sentiment of each document, asses based on the following axes.',
        '# Ideological Tilt (Liberal vs. Conservative)', 
        '- Gauge where the document stands on the traditional left-right political spectrum (e.g., social welfare spending, cultural values, role of government).',
        '- -5 = Highly Conservative, 0 = Centrist/Moderate, +5 = Highly Liberal',
        '# Immigration Approach (Closed vs. Open Borders)',
        '- Reflects how permissive or restrictive the document is regarding immigration, visas, refugees, and border enforcement.',
        '- -5 = Strictly Restrictive (Closed), 0 = Balanced/Selective, +5 = Extremely Open',
        '# Executive Power (Restriction vs. Expansion)',
        '- Measures whether the document curtails presidential power (by imposing checks or limiting discretion) or broadens the executive\'s reach (allowing unilateral actions or emergency powers).',
        '- -5 = Strongly Restrains Executive Authority, 0 = Neutral/Balanced, +5 = Strongly Expands Executive Authority',
        '# Federalism (State vs. Federal Primacy)',
        '- Evaluates if the documents shifts power and decision-making downward to states or upward to the federal government. A high positive score suggests centralizing authority in Washington, while a negative score favors decentralized/state-led governance.',
        '- -5 = Heavy State Control, 0 = Cooperative Balance, +5 = Strong Federal Control',
        '# Security vs. Liberty (National Security vs. Civil Liberties)',
        '- Captures whether the document prioritizes national security and law enforcement powers (potentially at civil liberties\' expense) or emphasizes individual rights and freedoms (possibly limiting security measures).',
        '- -5 = Strong Liberties/Privacy Focus, 0 = Balanced Approach, +5 = Heavy Security Focus',
        '# Economic Regulation (Deregulation vs. Intervention)',
        '- Determines how deeply the document involves the government in economic matters—whether it rolls back rules and fosters free markets or imposes policies (e.g., wage/price controls, mandatory guidelines) to steer the economy.',
        '- -5 = Strong Deregulation/Laissez-Faire, 0 = Moderate Regulation, +5 = Heavy Government Intervention',
        '# Social Inclusion (Conformity vs. Diversity)',
        '- Assesses whether the document promotes inclusivity (racial, ethnic, religious, gender, LGBTQ+ protections) and encourages pluralism, or whether it trends toward uniform standards, fewer cultural accommodations, and more traditional/conforming policies.'
        '-5 = Emphasis on Conformity/Traditional Uniformity, 0 = Neutral, +5 = Emphasis on Diversity/Pluralism',
        '{text}',
    ])
)

def political_bias_pipeline(model):
    return political_bias_prompt | model.with_structured_output(PoliticalBiasResponse, include_raw=True)

In [10]:
source_dir = 'data/executive_orders/cleaned/'
target_dir = 'data/executive_orders/political_bias/'
log_path = 'data/executive_orders/political_bias_log.csv'

pipeline = political_bias_pipeline(gemini)

def process_eo(eo_path):
    stats = {
        'path': eo_path
    }

    start_time = time.time()
    try:
        with open(source_dir + eo_path) as source_file:
            eo = json.load(source_file)
            result = pipeline.invoke({'text': '\n'.join(eo["content"])})
            stats = stats | build_stats(result)

        # write the cleaned eo to the target directory
        with open(target_dir + eo_path, 'w') as target_file:
            json.dump(result['parsed'].model_dump(), target_file, indent=4)
    except Exception as e:
        stats['error'] = str(e)

    duration = time.time() - start_time
    stats['duration'] = duration

    return stats

In [12]:
concurrency = 8

eo_paths = os.listdir(source_dir)
eo_paths.sort()

results = []
with ThreadPoolExecutor(max_workers=concurrency) as executor:
    futures = [
        executor.submit(process_eo, path) for path in eo_paths[0:200]
    ]

    for future in as_completed(futures):
        result = future.result()
        print(result)
        results.append(result)

pandas.DataFrame(results).to_csv(log_path, index=False)

{'path': 'commission-appointing-commissioners-for-surveying-the-district-territory-for-the-permanent.json', 'input_tokens': 1057, 'output_tokens': 85, 'duration': 1.2045845985412598}
{'path': 'executive-order-1.json', 'input_tokens': 802, 'output_tokens': 80, 'duration': 1.2111272811889648}
{'path': 'executive-order-0.json', 'input_tokens': 802, 'output_tokens': 98, 'duration': 1.3281846046447754}
{'path': 'executive-order-1000-alamo-national-forest-new-mexico.json', 'input_tokens': 1671, 'output_tokens': 84, 'duration': 1.352602481842041}
{'path': 'executive-order-10.json', 'input_tokens': 844, 'output_tokens': 115, 'duration': 1.3770229816436768}
{'path': 'executive-order-100.json', 'input_tokens': 1068, 'output_tokens': 90, 'duration': 1.4722206592559814}
{'path': 'executive-order-10000-regulations-governing-additional-compensation-and-credit-granted.json', 'input_tokens': 5313, 'output_tokens': 105, 'duration': 1.4861409664154053}
{'path': 'executive-order-10002-exemption-lawrence-